# Import des modules

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from ast import literal_eval
import csv

# Création des variables

In [2]:
DF0 = pd.read_csv("./../Tables/ReferentielNoms.csv",converters={"FormesNonNormalisées": literal_eval}).set_index("NomNormalisé")
DF0

,Types,FormesNonNormalisées,RenvoiToponymique
NomNormalisé,,,
Anne,['patronymic'],"[annen, Anne]",NaN
Enderlin,['patronymic'],"[enderlin, Enderlin]",NaN
Greden,[],"[greden, Greden]",NaN
"aarau, von",['toponymic'],"[von arowe, aarau, von]",Aarau
achenheim,['toponymic'],[achenheim],Achenheim
...,...,...,...
ölmann,['occupational'],"[oleman, ölmann, olman]",NaN
ösenwoog,['toponymic'],"[ösenwog, ösenwoog]",NaN
österreicher,['toponymic'],"[estericher, österreicher]",Autriche


# Création de la soupe

In [3]:
soup = BeautifulSoup(open('./../../base stras/aves_aa_195_p1.xml', 'r'))

# Création de la liste des éléments de la soupe à comparer

In [4]:
ListeNomsDoc = []
for surname in soup.find_all('surname'):
    ListeNomsDoc.append(surname.get_text().lower())
for addName in soup.find_all('addname'):
    ListeNomsDoc.append(addName.get_text().lower())
print (ListeNomsDoc)

['zorn', 'melbrü', 'von mulnheim', 'von landsperg', 'spiegel', 'vener', 'mösüng', 'hüffel', 'zum rüste', 'von mulnheim', 'meyenwarte', 'spiegel', 'nöppe', 'manse', 'zorne', 'von mulnheim', 'bockel', 'büman', 'mürsell', 'hesse', 'hesse', 'von mulnheim', 'von mulnheim', 'bock', 'bock', 'bock', 'büshsener', 'bock', 'büman', 'bock', 'bock', 'mösung', 'spender', 'voltsche', 'berer', 'berer', 'büman', 'lentzel', 'lentzel', 'von reßheim', 'bürgraüve', 'von endingen', 'hümbrecht', 'elhart', 'elhart', 'elhart', 'von kagenecke', 'merswin', 'lentzel', 'burggraffe', 'huffell', 'swöpp', 'von güdertheim', 'ritter', 'blenckel', 'zür megde', 'rössel', 'lentzel', 'schulteiß', 'schulteiße', 'von g[...]', 'zorn', 'fülle', 'bocke', 'büman', 'büman', 'schalck', 'armbruster', 'armbruster', 'lumbart', 'lumbart', 'ammelüng', 'pfie', 'luteßheim', 'tesche', 'von nothalden', 'berner', 'merckels', 'juden', 'hagen', 'toppeler', 'swartz', 'toppeler', 'waltenheim', 'zür glocken', 'rückorb', 'wernlin', 'erstheim', 'e

# Comparaison de la liste de la soupe à notre dataframe

In [5]:
for element in ListeNomsDoc :
    
    #Création des listes de références à partir du dataframe DF1.
    Colonne_FormesNonNormalisées = DF0['FormesNonNormalisées'].tolist()
    Colonne_NomNormalisé = DF0.index.tolist()
    Colonne_Types = DF0['Types'].tolist()
    
    #Vérification de l'existence de la forme non normalisée dans la liste des listes de formes non-normalisées.
    CheckSurnameNN = any(element in sublist for sublist in Colonne_FormesNonNormalisées)
    DicoPersonne = {}

    #Sinon, on demande un str en entrée pour le comparer à la liiste des formes normalisées.
    if CheckSurnameNN != True :
        SurnameN = input('Entrez la forme normalisée du nom ' + element + ' :')
        
        #Si l'entrée figure dans la liste des prénoms normalisées, on ajoute la forme non normalisée à la liste de celles-ci dans le dataframe.
        if SurnameN in Colonne_NomNormalisé:
            CelluleModif = DF0.loc[SurnameN, 'FormesNonNormalisées']
            CelluleModif.append(element)
            
        #Sinon, on crée un dataframe qu'on additionnera à l'existant.
        else :
            
            #On crée le couple 'PrénomNormalisé' dont la valeur correspond à la forme la plus courante du prénom.
            DicoPersonne['NomNormalisé'] = SurnameN
            
            #On crée le couple 'Types' dont la valeur est une liste contenant des tags proposant une description typologique du prénom.
            typologie = []
            n = int(input('Entrez le nombre de types à attribuer au nom ' + element + ' :'))
            for i in range(0,n):
                Type = input('Entrez un des types à attribuer au nom ' + element + ' :')
                typologie.append(Type)
            DicoPersonne['Types'] = typologie
            
            #On crée le couple 'FormesNonNormalisées' dont la valeur est une liste contenant toutes les formes alternatives du prénom.
            ListeNN = []
            ListeNN.append(element)
            if element != SurnameN :
                ListeNN.append(SurnameN)
            DicoPersonne['FormesNonNormalisées'] = ListeNN
            
            #On crée le couple 'RenvoiToponymique' dont la valeur est le nom normalisé en français (si possible) du lieu.
            for Type in typologie:
                if Type == 'toponymic':
                    RenvoiToponymique = input('Entrez le nom normalisé, en français si possible, du lieu auquel' + element + ' fait référence : ')
                    DicoPersonne['RenvoiToponymique'] = RenvoiToponymique
            
            #On crée le dataframe à ajouter à l'existant.
            DF1 = pd.DataFrame([DicoPersonne]).set_index('NomNormalisé')
            
            #On concatène le dictionnaire obtenu à notre Referentiel avant de recommencer toute l'opération.
            DF0 = pd.concat([DF0,DF1])
DF0

Entrez la forme normalisée du nom zum stangke :stange, zum
Entrez le nombre de types à attribuer au nom zum stangke :1
Entrez un des types à attribuer au nom zum stangke :toponymic
Entrez le nom normalisé, en français si possible, du lieu auquelzum stangke fait référence : 
Entrez la forme normalisée du nom zum haven :hafen, zum
Entrez le nombre de types à attribuer au nom zum haven :1
Entrez un des types à attribuer au nom zum haven :toponymic
Entrez le nom normalisé, en français si possible, du lieu auquelzum haven fait référence : 
Entrez la forme normalisée du nom von rosenwiler :rosenwiller, von
Entrez le nombre de types à attribuer au nom von rosenwiler :1
Entrez un des types à attribuer au nom von rosenwiler :toponymic
Entrez le nom normalisé, en français si possible, du lieu auquelvon rosenwiler fait référence : Rosenwiller
Entrez la forme normalisée du nom zü der glocken :glocken, zur
Entrez la forme normalisée du nom züm helffan :elefant, zum
Entrez le nombre de types à attri

,Types,FormesNonNormalisées,RenvoiToponymique
NomNormalisé,,,
Anne,['patronymic'],"[annen, Anne]",NaN
Enderlin,['patronymic'],"[enderlin, Enderlin]",NaN
Greden,[],"[greden, Greden]",NaN
"aarau, von",['toponymic'],"[von arowe, aarau, von]",Aarau
achenheim,['toponymic'],[achenheim],Achenheim
...,...,...,...
üsenberg,['toponymic'],"[uͦsemberg, üsemberg, üsenberg]",Üsenberg
"stange, zum",[toponymic],"[zum stangke, stange, zum]",
"hafen, zum",[toponymic],"[zum haven, hafen, zum]",


In [7]:
DF0 = DF0.sort_values(by=['NomNormalisé'])
DF0.to_csv("./../Tables/ReferentielNoms.csv")